In [1]:
import sys
import os
import time

sys.path.append(os.path.abspath("../.."))

### Test 1


In [ ]:
from tiny_graph.graph.executable import Graph
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import History, LastValue
from tiny_graph.checkpoint.storage.local_storage import LocalStorage
from tiny_graph.constants import START, END
from typing import Dict


# Define our state model
class ProcessState(GraphState):
    status: LastValue[str]
    results: History[Dict[str, float]]


# Initialize state and graph with local storage and chain_id
chain_id = "process_workflow_v1"
state = ProcessState(status="", results={})
storage = LocalStorage()
graph = Graph(state=state, checkpoint_storage=storage, chain_id=chain_id)


# Define processing nodes
@graph.node()
def initialize_process(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "initializing"}


@graph.node()
def process_data_1(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_1", "results": {"accuracy": 0.85, "step": 1.0}}


@graph.node(interrupt="after")
def process_data_2(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_2", "results": {"accuracy": 0.92, "step": 2.0}}


@graph.node()
def finalize(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "completed"}


# Create the workflow
graph.add_edge(START, "initialize_process")
graph.add_edge("initialize_process", "process_data_1")
graph.add_edge("process_data_1", "process_data_2")
graph.add_edge("process_data_2", "finalize")
graph.add_edge("finalize", END)

# Compile and execute
graph.compile()
graph.visualize()

In [ ]:
graph.execute()

In [ ]:
storage.list_checkpoints(chain_id)

graph.checkpoint_storage.list_checkpoints(chain_id)

In [ ]:
storage.list_checkpoints(chain_id)

### Test 2


In [2]:
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import History
from tiny_graph.checkpoint.local_storage import LocalStorage
from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END


class StateForTestWithHistory(GraphState):
    execution_order: History[str]


state = StateForTestWithHistory(execution_order=[])
storage = LocalStorage()
graph = Graph(state=state, checkpoint_storage=storage)


@graph.node()
def task1(state):
    print("task1")
    time.sleep(0.5)
    return {"execution_order": "task1"}


@graph.node()
def task2(state):
    print("task2")
    time.sleep(0.5)
    return {"execution_order": "task2"}


@graph.node()
def task3(state):
    print("task3")
    time.sleep(1)
    return {"execution_order": "task3"}


@graph.node()
def task4(state):
    print("task4")
    time.sleep(2)
    print("task4 done")

    return {"execution_order": "task4"}


@graph.node()
def task5(state):
    print("task5")
    time.sleep(1)
    return {"execution_order": "task5"}


@graph.node(interrupt="before")
def task6(state):
    print("task6")
    return {"execution_order": "task6"}


graph.add_edge(START, "task1")
graph.add_edge("task1", "task2")
graph.add_edge("task2", "task3")
graph.add_edge("task2", "task4")
graph.add_edge("task2", "task5")
graph.add_edge("task4", "task6")
graph.add_edge("task3", "task6")
graph.add_edge("task5", "task6")
graph.add_edge("task6", END)
graph.compile()

graph.visualize()

DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.viewing.view(['quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.quote(['is_html_string', 'is_valid_id', 'dot_keywords', 'endswith_odd_number_of_backslashes', 'escape_unescaped_quotes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.a_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.attr_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.dot.Dot.clear(['keep_attrs'])
DEBUG:graphviz._tools:deprecate posit

In [3]:
chain_id = graph.start()
print(chain_id)
assert all(
    task in graph.state.execution_order
    for task in ["task1", "task2", "task3", "task4", "task5"]
), "tasks are not in there"
assert len(storage.list_checkpoints(graph.chain_id)) == 4  # n + 1 due to interrupt

DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
DEBUG:tiny_graph.graph.executable:Executing task in node: task1


task1


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_e3e0c2d8-eb7c-4a89-8b63-c1a3831c2f98' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task1
DEBUG:tiny_graph.graph.executable:Executing task in node: task2


task2


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_51118c37-ed5f-4101-bca5-0460b4dd2e91' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task2
DEBUG:tiny_graph.graph.executable:Executing task in node: task5
DEBUG:tiny_graph.graph.executable:Executing task in node: task4
DEBUG:tiny_graph.graph.executable:Executing task in node: task3


task5
task4
task3


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_5268979f-6170-47e5-aa10-e8d2112aab83' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: group_task5_task4_task3
DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.PAUSE
INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_c4bf7f9a-c5bd-4c87-89e6-85c8e02f44ed' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task6


task4 done
chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d


In [4]:
graph.state.execution_order

['task1', 'task2', 'task5', 'task3', 'task4']

In [5]:
storage.list_checkpoints(graph.chain_id)

[Checkpoint(checkpoint_id='checkpoint_e3e0c2d8-eb7c-4a89-8b63-c1a3831c2f98', chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d', chain_status=<ChainStatus.RUNNING: 3>, state_class='__main__.StateForTestWithHistory', state_version='a68afb7bed1b464bc9ee75fa7adf0b26', data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1"]}', timestamp=datetime.datetime(2024, 11, 27, 22, 8, 56, 977937), next_execution_node=None, last_executed_node=None, executed_nodes={'task1'}),
 Checkpoint(checkpoint_id='checkpoint_51118c37-ed5f-4101-bca5-0460b4dd2e91', chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d', chain_status=<ChainStatus.RUNNING: 3>, state_class='__main__.StateForTestWithHistory', state_version='a68afb7bed1b464bc9ee75fa7adf0b26', data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1","task2"]}', timestamp=datetime.datetime(2024, 11, 27, 22, 8, 57, 509772), next_execution_node=None, last_executed_node=None, executed_nodes={'task2', 'task1'}),

In [6]:
graph.state

StateForTestWithHistory(version='a68afb7bed1b464bc9ee75fa7adf0b26', execution_order=['task1', 'task2', 'task5', 'task3', 'task4'])

In [7]:
# start a new chain just to test the load from checkpoint
new_chain_id = graph.start()
print(new_chain_id)

DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
DEBUG:tiny_graph.graph.executable:Executing task in node: task1


task1


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_d11742d3-ad17-470e-ba9b-119ad8d1df0f' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task1
DEBUG:tiny_graph.graph.executable:Executing task in node: task2


task2


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_735c824d-12a0-4ea1-a79c-fff5fca870cd' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task2
DEBUG:tiny_graph.graph.executable:Executing task in node: task5
DEBUG:tiny_graph.graph.executable:Executing task in node: task4
DEBUG:tiny_graph.graph.executable:Executing task in node: task3


task5
task4
task3


INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_717faf91-e0d8-4689-893e-0f24a725a1ca' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: group_task5_task4_task3
DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.PAUSE
INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_dce8d7b0-aff7-4315-89cf-1badc8c55e58' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task6


task4 done
chain_72d49813-4344-45dc-871b-68fa6d14aba3


In [8]:
from rich import print as rprint

rprint(storage._storage)

defaultdict(<class 'dict'>, {
    'chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d': {
        'checkpoint_e3e0c2d8-eb7c-4a89-8b63-c1a3831c2f98': Checkpoint(
            checkpoint_id='checkpoint_e3e0c2d8-eb7c-4a89-8b63-c1a3831c2f98',
            chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d',
            chain_status=<ChainStatus.RUNNING: 3>,
            state_class='__main__.StateForTestWithHistory',
            state_version='a68afb7bed1b464bc9ee75fa7adf0b26',
            data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1"]}',
            timestamp=datetime.datetime(2024, 11, 27, 22, 8, 56, 977937),
            next_execution_node=None,
            last_executed_node=None,
            executed_nodes={'task1'}
        ),
        'checkpoint_51118c37-ed5f-4101-bca5-0460b4dd2e91': Checkpoint(
            checkpoint_id='checkpoint_51118c37-ed5f-4101-bca5-0460b4dd2e91',
            chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d',
            chain_status=<ChainStatus.RUNNING: 3>,
            state_class='__main__.StateForTestWithHistory',
            state_version='a68afb7bed1b464bc9ee75fa7adf0b26',
            data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1","task2"]}',
            timestamp=datetime.datetime(2024, 11, 27, 22, 8, 57, 509772),
            next_execution_node=None,
            last_executed_node=None,
            executed_nodes={'task2', 'task1'}
        ),
        'checkpoint_5268979f-6170-47e5-aa10-e8d2112aab83': Checkpoint(
            checkpoint_id='checkpoint_5268979f-6170-47e5-aa10-e8d2112aab83',
            chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d',
            chain_status=<ChainStatus.RUNNING: 3>,
            state_class='__main__.StateForTestWithHistory',
            state_version='a68afb7bed1b464bc9ee75fa7adf0b26',
            data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1","task2","task5","task3",
"task4"]}',
            timestamp=datetime.datetime(2024, 11, 27, 22, 8, 59, 525994),
            next_execution_node=None,
            last_executed_node=None,
            executed_nodes={'task3', 'task1', 'task5', 'task4', 'task2'}
        ),
        'checkpoint_c4bf7f9a-c5bd-4c87-89e6-85c8e02f44ed': Checkpoint(
            checkpoint_id='checkpoint_c4bf7f9a-c5bd-4c87-89e6-85c8e02f44ed',
            chain_id='chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d',
            chain_status=<ChainStatus.PAUSE: 2>,
            state_class='__main__.StateForTestWithHistory',
            state_version='a68afb7bed1b464bc9ee75fa7adf0b26',
            data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1","task2","task5","task3",
"task4"]}',
            timestamp=datetime.datetime(2024, 11, 27, 22, 8, 59, 536991),
            next_execution_node='task6',
            last_executed_node=None,
            executed_nodes={'task3', 'task1', 'task5', 'task4', 'task2'}
        )
    },
    'chain_72d49813-4344-45dc-871b-68fa6d14aba3': {
        'checkpoint_d11742d3-ad17-470e-ba9b-119ad8d1df0f': Checkpoint(
            checkpoint_id='checkpoint_d11742d3-ad17-470e-ba9b-119ad8d1df0f',
            chain_id='chain_72d49813-4344-45dc-871b-68fa6d14aba3',
            chain_status=<ChainStatus.RUNNING: 3>,
            state_class='__main__.StateForTestWithHistory',
            state_version='a68afb7bed1b464bc9ee75fa7adf0b26',
            data='{"version":"a68afb7bed1b464bc9ee75fa7adf0b26","execution_order":["task1"]}',
            timestamp=datetime.datetime(2024, 11, 27, 22, 9, 16, 669239),
            next_execution_node=None,
            last_executed_node=None,
            executed_nodes={'task1'}
        ),
        'checkpoint_735c824d-12a0-4ea1-a79c-fff5fca870cd': Checkpoint(
            checkpoint_id='checkpoint_735c824d-12a0-4ea1-a79c-fff5fca870cd',
            chain_id='chain_72d49813-4344-45dc-871b-68fa6d14aba3',
            chain_status=<ChainStatus.RUNNING: 3>,
            state_class='__main__.Stat

In [9]:
print("current_chain_id", graph.chain_id)
print("saved_chain_id", chain_id)
graph.load_from_checkpoint(chain_id)
print("after load chain_id", graph.chain_id)

graph.resume()
assert all(
    task in graph.state.execution_order
    for task in ["task1", "task2", "task3", "task4", "task5", "task6"]
)

DEBUG:tiny_graph.graph.executable:Loaded checkpoint checkpoint_c4bf7f9a-c5bd-4c87-89e6-85c8e02f44ed for chain chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d
DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_943c8f55-1e45-405d-a000-52eef1382995' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task1
INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_9057ef53-86eb-49af-8535-8ff188a459f8' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: task2
INFO:tiny_graph.checkpoint.local_storage:Checkpoint 'checkpoint_0d059819-9936-424f-b808-98e8ebcdc8b4' saved in memory.
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: group_task5_task4_task3
DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
DEBUG:tiny_graph.graph.executable:Executing task in node: task6
INFO:tiny_graph.checkpoint.local_storage:Ch

current_chain_id chain_72d49813-4344-45dc-871b-68fa6d14aba3
saved_chain_id chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d
after load chain_id chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d
task6


In [10]:
graph.load_from_checkpoint(chain_id)
graph.state.execution_order

DEBUG:tiny_graph.graph.executable:Loaded checkpoint checkpoint_0d141931-41a0-4c63-8f65-0b971aeda57d for chain chain_91eaafb1-3a29-4b2d-98a1-feb01ab7ad8d


['task1', 'task2', 'task5', 'task3', 'task4', 'task6']

In [11]:
graph.state

StateForTestWithHistory(version='a68afb7bed1b464bc9ee75fa7adf0b26', execution_order=['task1', 'task2', 'task5', 'task3', 'task4', 'task6'])

In [12]:
getattr(graph.state, "execution_order")

['task1', 'task2', 'task5', 'task3', 'task4', 'task6']